In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import os
import pandas as pd

colab_path = '/content/drive/My Drive/NLP-SentimentAnalysis/'
base_path=colab_path+'aclImdb'
processed_data_path=colab_path+'processed_data'

train_data_path = base_path + '/train'
test_data_path = base_path + '/test'

def read_data(path,sentiment):
    dic={'text':[],'label':[]}
    for file_path in os.listdir(path):
        with open(os.path.join(path,file_path),encoding='utf8') as f:
            text=f.readline()
        dic['text'].append(text)
        dic['label'].append(sentiment)
    df=pd.DataFrame(dic)
    return df

def get_data(raw_data_path,saved_path):
    if os.path.exists(saved_path):
        df=pd.read_csv(saved_path)
    else:
        df_pos=read_data(raw_data_path+'/pos',1)
        df_neg=read_data(raw_data_path+'/neg',0)
        df = pd.concat([df_pos,df_neg],ignore_index=True)
        df.to_csv(saved_path,index=False)
    return df

In [3]:
df_train = get_data(train_data_path,processed_data_path + '/train.csv')

In [4]:
df_test = get_data(test_data_path,processed_data_path+'/test.csv')

In [5]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
logistic_regressor = LogisticRegression()
tf_idf_vectorizer = TfidfVectorizer()
pipeline = Pipeline([
    ('vectorizer',tf_idf_vectorizer),
    ('model',logistic_regressor)
])
pipeline.fit(df_train['text'],df_train['label'])

Pipeline(memory=None,
         steps=[('vectorizer',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('model',
                 LogisticRegression(C=1.0, class_weight=None, dual=False,
                                    fit_intercept=True, intercept_

In [6]:
pred_train_label = pipeline.predict(df_train['text'])
from sklearn.metrics import accuracy_score as accuracy
print('Training accuracy',accuracy(pred_train_label,df_train['label']))

Training accuracy 0.93328


In [7]:
pred_test_label = pipeline.predict(df_test['text'])
print('Testing accuracy',accuracy(pred_test_label,df_test['label']))

Testing accuracy 0.88316
